# Deep Learning in Keras

## Prosta sieć przewidująca typ kwiatów

W tym notebooku stworzymy kilka prostych sieci neuronowych za pomocą biblioteki keras do wykrywania typu irysa

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import keras
print(keras.__version__)

Za pomocą pakietu pandas Załadujmy dane ze zbioru IRIS dataset:

In [ ]:
...

Przeanalizujmy zestaw danych:
* informacjami o zmiennych
* statystykami opisowymi

In [ ]:
...

In [ ]:
...

Jak rozkładają się różne klasy kwiatów w tym zbiorze?

In [ ]:
...

Żeby wygodnie korzystać z modeli uczenia maszynowego w pythonie podziel kolumny ramki na predyktory X i zmienną niezależną :

In [ ]:
X = iris.drop('y', axis=1)
y = iris['y'].values

Jak wygląda X?

Jak wygląda y?

Jak widać, etykiety do naszego zbioru są wektorem z przypisanymi numerami kategorii. Aby zamienić taką zmienną kategorialną na format odpowiedni dla kerasa, należy dokonać procesu kodowania 'one-hot' encoding. Nasze etykiety będą teraz reprezentowane przez binarną macierz, w której każda kolumna będzie odpowiadała jednej kategorii. Możemy to zrobić na kilka sposobów:

In [ ]:
# sposób1 - na bazie macierzy jednostkowej
y_onehot = np.eye(3)[y.astype('int')]
print(y_onehot)

In [ ]:
# sposób2 - z indeksowaniem
y_onehot = np.zeros((150,3))
y_onehot[np.arange(150), y.astype('int')] = 1
print(y_onehot)

In [ ]:
# sposób3 - z pomocą Kerasa
from keras.utils import np_utils
y_onehot = np_utils.to_categorical(y, 3)
print(y_onehot)

Podzielmy teraz losowo nasz zbiór na uczący i testowy za pomocą wbudowanych funkcji sklearna. Niech zbiór testowy zawiera 20% całego zbioru obserwacji

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(...)

print('X_train shape:{}'.format(X_train.shape))
print('X_test shape:{}'.format(X_test.shape))

print('y_train shape:{}'.format(y_train.shape))
print('y_test shape:{}'.format(y_test.shape))

Zaimportujmy Sequential będące modelem sieci neuronowej w Kerasie, oraz Dense -  podstawową warstwę ukrytą

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

Zaininicjujmy obiekt klasy Sequential. Następnie za pomocą metody `.add` będziemy dodawać kolejne warstwy ukryte i końcową

In [ ]:
model = Sequential()

Dodajmy do naszej sieci warstwę typu Dense. W pierwszej warstwie należy wyspecyfikować `input_dim` liczbę zmiennych będących predyktorami. Ponadto należy podać liczbę neuronów `units`, oraz funkcję aktywacji dla tej warstwy `activation`. Jako funkcję aktywacji podajmy *Rectified Linear Unit* - `relu`

In [ ]:
model.add(
    Dense(units=50, input_dim=4, activation='relu')
)

Dodajmy teraz warstwę składającą się z 50 neuronów, z taką samą funkcją aktywacji

In [ ]:
model.add(
    Dense(units=100, activation='relu')
)

Na sam koniec dodajmy warstwę końcową z trzema neuronami, w której każdy neuron będzie przedstawiał prawdopobieństwo wystąpienia danej klasy. Z jakiej funkcji aktywacji skorzystamy?

In [ ]:
model.add(
    Dense(units=3,...?)
)

Mamy teraz sieć neuronową składającą się z dwóch warstw ukrytych i jednej wyjściowej. Sprawdźmy sobie podsumowanie dla tego modelu:

In [ ]:
print(model.summary())

To samo możemy zrobić w formie graficznej. Pamiętaj o zainstalowaniu pydot i graphviz:

In [ ]:
from keras.utils import plot_model
plot_model(
    model,
    to_file='model.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='LR'
)

In [ ]:
# przypadku gdy nie mamy zainstalowanego, to instalujemy pakiet graphviz, i zmieniamy zmienną PATH w systemie.
# Możemy zmienić ją także z poziomu pythona
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

Możemy otworzyć tak stworzony obraz na dysku bądź z poziomu jupyter notebooka:

In [ ]:
from IPython.display import Image
Image(filename='model.png')

Po zdefiniowaniu architektury sieci, za pomocą `.compile` należy skonfigurować taki model do procesu uczenia. Najważniejsze parametry, jakie trzeba zdefiniować to:
* `loss` - funkcja straty
* `optimizier` - rodzaj optymalizatora
* `metrics` - lista metryk do ewaluacji

In [ ]:
model.compile(
    optimizer='adam',
    metrics=['accuracy'],
    loss='categorical_crossentropy'
)

Optymalizator może być stringiem z zakresu dostępnych w bilbiotece. Może być też obiektem z keras.optimizers

In [ ]:
from keras import optimizers

opt = optimizers.Adam(lr=0.001)
model.compile(
    optimizer=opt,
    metrics=['accuracy'],
    loss='categorical_crossentropy'
)

Po skonfigurowaniu modelu możemy przejść do trenowania za pomocą metody `.fit`

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=150
)

Jak nasza sieć działa na zbiorze, który nie brał udziału w procesie uczenia?

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(loss)
print(accuracy)

Jeśli chcielibyśmy dokonać predykcji dla konkretnej obserwacji, korzystamy z funkcji `.predict`. Metoda przyjmuje macierz 2D o kształcie (liczba obserwacji, liczba cech):

In [ ]:
print(X_train.iloc[0].shape)
print(np.expand_dims(X_train.iloc[0], axis=0).shape)

In [ ]:
np.set_printoptions(suppress=True)
predictions = model.predict(...)
print(predictions)
# jak programistycznie wyciągnąć numer klasy z najwyższym prawdopobieństwem

In [ ]:
plt.bar(x=[1,2,3], height=np.squeeze(predictions))
plt.title('Prawdopobieństwo wystąpienia danej klasy dla danego przypadku \n')

Analizując obiekt `history` możemy zobaczyć jak wyglądała skuteczność treningowa podczas procesu uczenia:

In [ ]:
plt.plot(history.history['acc'])
plt.title('Skuteczność na danych treningowych')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

Gdybyśmy chcieli zobaczyć jak w procesie uczenia sieć dopasowała wagi, tak wygląda to dla pierwszej warstwy:

In [ ]:
model.layers[0].get_weights()

## Sieć przewidująca typ kwiatów - praca własna

#### Zadanie do samodzielnego rozwiązania

Stwórz sieć neuronową mającą 4 warstwy:
* Warstwa 50 neuronów, funkcja aktywacji `tanh`
* Dwie warstwy po 100 neuronów z funkcją aktywacji `tanh`
* Warstwa z trzema neuronami, z funkcją aktywacji `softmax`

Do skonfigurowania `.compile`, wykorzystaj optymalizator `adam` z learning_rate na poziomie 0.0001, funkcja straty to `categorical_crossentropy`. 

do metody fit dodaj argument `validation_data=(X_test, y_test)` - pozwoli on na jednoczesną ewaluację modelu na danych walidacyjnych

Stwórz wykres przedstawiający skuteczność treningową i testową w zależności od numeru epoki.
Analizując obiekt historii do skuteczności treninowej dobierzesz się `history.history['acc']`, natomiast do skuteczności testowej `history.history['val_acc']`

## Sieć przewidująca wynik meczu piłkarskiego

#### Zadanie do samodzielnego wykonania
Poniżej wczytasz dane dotyczące historycznych spotkań piłkarskich w pierwszej lidze hiszpańskiej. Dla ułatwienia proces oczyszczania danych i feature engineeringu został już przeprowadzony.
W pliku `liga_hiszpanska.csv` znajdziesz wszystkie mecze dla sezonów od 2005 do 2016. Twoim zadaniem jest:
* Wczytaj dane
* Przeanalizuj zbiór (`.info, .describe, .head`)
* Zakoduj zmienną ['Outcome'] do postaci 'one-hot encoding'. Hint: przyjrzyj się funkcji `pd.get_dummies()`. Wynik zapisz do zmiennej `one_hot`
* Stwórz zbiór uczący X_train- to zbiór, w którym `Season` jest starszy niż 2015
* Stwórz zbiór validacyjny X_valid - to zbiór w którym `Season` to 2015
* Stwórz zbiór testowy X_test - to zbiór, w którym `Season` to 2016
Zmienne będące predyktorami znajdziesz w liście o nazwie `features` w komórce poniżej

Nie zapomnij dla każdego z tych zbiorów stworzyć odpowiadających im macierzy etykiet y_train, y_test, y_valid

Przykładowe stworzenie zbioru treningowego 

`X_train = dane.loc[dane['Season'] <= 2014, features]`

`y_train = one_hot.loc[dane['Season'] <= 2014, ].values`

* Za pomocą `StandardScaler` z pakietu `scikit-learn` dokonaj standaryzacji na zbiorze uczącym . Ten sam obiekt klasy StandardScaler wykorzystaj do zestandaryzowania X_test i X_valid

* Następnie stwórz model sieci neuronowej, architekturę wybierz dowolnie. Pamiętaj, że w przypadku przeuczenia modelu, możesz spróbować dodać regularyzację typu Dropout po każdej warstwie `Dense`, w taki sposób: `model.add(Dropout(rate=?))`
* Po zdefiniowaniu architektury sieci, uruchom proces trenowania.

* Wykreśl przebieg skuteczności uczącej i walidacyjnej w czasie
* Na samym końcu dokonaj ewaluacji modelu na danych ze zbioru testowego - takiego, który wcześniej nie brał udziału w procesie uczenia
* Sprawdź jak Twoja sieć neuronowa działa w porównaniu z klasyfikatorem kNN, oraz z klasyfikatorem naiwnym - tj obstaw, że we wszystkich przypadkach wygrają gospodarcze meczu. hint: żeby z 'one-hot encoding' przejść do numerów klas w jednym wektorze wykorzystaj `y_test.argmax(axis=-1)`

## Konwolucyjna sieć neuronowa - MNIST

Tym razem wytrenujemy sieć konwolucyjną do rozpoznawania ręcznie zapisanych cyfr z obrazów - popularny zbiór MNIST.

Pobierzmy odpowiednie dane z wbudowanej funkcji:

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Ile mamy obserwacji?

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Narysujmy pierwszą wybraną obserwację ze zbioru uczącego:

In [ ]:
X_train[0]

In [ ]:
plt.imshow(X_train[0], cmap='gray')

Aby dostosować format danych do przetwarzania przez Kerasa, wymiary naszego zbioru muszą mieć następującą postać:
`(liczba obserwacji, liczba kanałów obrazu, wysokość, szerokość)`. Jest to tzw format 'channels_first'

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test= X_test.reshape(X_test.shape[0], 1, 28, 28)

Wymiary po rozszerzeniu:

In [ ]:
print(X_train.shape)
print(X_test.shape)

Normalizujemy dane do zakresu [0,1].

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Tak jak w poprzednim przypadku musimy zastosować 'one-hot encoding' dla naszych etykiet. Wykorzystaj np_utils

In [ ]:
Y_train = ...
Y_test = ...

Załadujmy odpowiednie rodzaje warstw do przetwarzania obrazów:
* `Flatten` jest warstwą, która spłaszcza dane
* `Convolution2D` jest warstwą konwolucyjną. Pierwszy parametr to liczba filtrów w warstwie, a drugi to krotka (x,x) oznaczająca wielkość filtra. W pierwszej warstwie zdefiniujemy też format danych - będzie to `channels_first`, i odpowiadająca formatowi (1,28,28). W przypadku wariantu `channel_last` byłoby to (28, 28, 1)
* `MaxPooling2D` - to warstwa poolingu, która dla zadanej wielkości okna `pool_size` wybiera wartość największą, dzięki czemu zmniejsza wymiar danych
* `Dropout` to warstwa regularyzacyjna. Przyjmuje argument `rate`, który oznacza jaka frakcja neuronów jest zamrażana w każdym kolejnym przejściu danych przez sieć (*forward pass*)

In [ ]:
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D

In [ ]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(1,28,28), data_format='channels_first'))
model.add(Convolution2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(10, activation='softmax'))

Ile parametrów będziemy 'uczyć'?

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(
    X_train,
    Y_train, 
    batch_size=32,
    nb_epoch=10
)

Po wytrenowaniu sieci, możemy zwizualizować wyuczone filtry. Najpierw pobierzmy wszystkie wartości wag z pierwszej warstwy konwolucyjnej

In [ ]:
layer = model.get_layer(... - podaj nazwę pierwszej warstwy konwolucyjnej)
weights = layer.get_weights()[0]
biases = layer.get_weights()[1]
weights.shape

 Usuńmy zbędny wymiar za pomocą funkcji `np.squeeze`

In [ ]:
weights_reshaped = np.squeeze(weights)

I narysujmy filtry wyuczone przez sieć

In [ ]:
plt.figure(figsize=(6,6))
for i in range(32):
    plt.subplot(6, 6, 1 + i)
    plt.imshow(weights_reshaped[:, :, i], cmap='gray')
plt.show()

## Klasyfikacja - CIFAR10

#### Zadanie do samodzielnego rozwiązania
Zapoznaj się ze zbiorem danych CIFAR-10 https://www.cs.toronto.edu/~kriz/cifar.html

* Załaduj dane analogicznie jak w przypadku zbioru mnist `keras.datasets.cifar10`. Przeanalizuj kształt otrzymanych danych. Jest to channels_first, czy channels_last?
* Za pomocą kodu dostępnego niżej zwizualizuj pierwszych 25 obserwacji w zbiorze
* Przekształć zmienną prognozowaną za pomocą 'one-hot encoding'
* Zmień typ danych x_train, i x_test na float32
* Znormalizuj x_train i x_test do wartości z zakrsu 0-1 (podziel przez 255)
* Stwórz model sieci neuronowej, używając architektury zawartej w dołączonym pliku `cifar_architecture.png`
* Skonfiguruj model używając dowolnie wybranego optymalizatora
* Zacznij proces trenowania, przyjmując `batch_size=64`. Liczbę epok ustaw na 5 `epochs=15`. Pamiętaj o dodaniu zbioru walidacyjnego jako parametr metody `.fit`. Skorzystaj z dostępnego już fragmentu kodu - zapisuje on na dysku wagi, jeśli skuteczność walidacyjna się zwiększyła
* Pójdź na kawę. Jeśli proces będzie się przedłużał, to wyłącz działanie komórki, i zmniejsz dataset


In [ ]:
class_names = ['Airplane','Automobile','Bird', 'Cat','Deer','Dog','Frog','Horse','Ship','Truck']
plt.figure(figsize=(8,8))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i,])
    plt.xlabel(class_names[y_train[i][0]])
plt.show()

In [ ]:
# w razie potrzeby zmniejszenia zbioru danych:
mask = np.random.choice(50000, 6000, replace=False)
print(x_train[mask].shape)
print(y_train[mask].shape)

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
checkpoint = ModelCheckpoint(
    'best_weigths.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max'
)

model.fit(x_train, y_train,
            batch_size=64,
            epochs=15,
            validation_data=(x_test, y_test),
            shuffle=True,
         callbacks=[checkpoint])
